In [1]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
pip install gradio openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71

In [3]:
import gradio as gr
from openai import OpenAI
import openai
import pandas as pd

In [4]:
import os
APIKEY = ''
os.environ['OPENAI_API_KEY'] = APIKEY

In [5]:
FILE = '/content/drive/MyDrive/Colab Notebooks/ac_overview_3.csv'
df = pd.read_csv(FILE)
df.dropna(inplace=True)

# REVIEW_MOMO = '/content/drive/MyDrive/Colab Notebooks/momo_reviews_0713.csv'
REVIEW_MOMO = '/content/drive/MyDrive/Colab Notebooks/mobile01_715.xlsx'
reviews_momo = pd.read_excel(REVIEW_MOMO)
# reviews_momo.dropna(axis=0, how='all', inplace=True)
# print(reviews_momo)

PREDICT_MOBILE01 = '/content/drive/MyDrive/Colab Notebooks/mobile_brand_prediction.csv'
predict_mobile = pd.read_csv(PREDICT_MOBILE01)
predict_mobile.sort_values(by=['average_prediction_scores'], ascending=False, inplace=True)
# print(predict_mobile)

brand_name = pd.read_excel(REVIEW_MOMO, 'brand_name')
# print(brand_name)

In [6]:
client = OpenAI(api_key=os.environ.get("APIKEY"))

In [7]:
def fetch_product_info(product_id, df):
  product = df[df.id == product_id].to_dict(orient='records')
  if product:
    return product[0]
  else:
    return None

def chat_with_openai(prompt, messageList):
  keyword = '評價'
  if keyword in prompt:
    messageList.append({"role": "user", "content": f"{prompt}"})
    messageList.append({"role": "assistant", "content": f"請根據{get_abstract(reviews_dict, get_ask_brand(reviews_dict, brand_name, prompt))}提供資訊。"})
  else:
    messageList.append({"role": "user", "content": f"{prompt}，坪數以及房間大小不能更動"})
    messageList.append({"role": "assistant", "content": f"請在{df}中尋找符合使用者輸入條件的冷氣，提供一至三個選擇，並請優先推薦{predict_mobile}中排序在前的品牌。"})

  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messageList
  )

  assistant_message = response.choices[0].message.content
  messageList.append({"role": "assistant", "content": assistant_message})

  return assistant_message, messageList

def handle_chat(prompt, messageList, df):
  if prompt.startswith("ProductID:"):
    product_id = int(prompt.split(":")[1].strip())
    product_info = fetch_product_info(product_id, df)

    if product_info:
      response = f"品名: {product_info['title']}\nPrice: ${product_info['discount_price']}\n能源效率: {product_info['efficiency']}"
    else:
      response = "Sorry, the product ID you provided does not exist."
  else:
    response, messageList = chat_with_openai(prompt, messageList)
  return response, messageList


In [8]:
def parse_user_query(query):
  keywords = {
    '坪': 'ping',
    '能效': 'efficiency',
    '能源效率': 'efficiency',
    '價格': 'discount_price',
    '價位': 'discount_price',
    '萬': 'discount_price'
  }

  detected_keywords = {kw: col for kw, col in keywords.items() if kw in query}
  return detected_keywords

# Example user query
query = "我想要知道能效高和價格便宜的空調"
detected_keywords = parse_user_query(query)
print(detected_keywords)


{'能效': 'efficiency', '價格': 'discount_price'}


In [9]:
def filter_and_sort_data(df, detected_keywords):
  # If no relevant keywords found, return empty
  if not detected_keywords:
    return pd.DataFrame()

  # Create a list to store sorting criteria
  sort_criteria = []

  if 'efficiency' in detected_keywords.values():
    sort_criteria.append('efficiency')
  if 'discount_price' in detected_keywords.values():
    sort_criteria.append('discount_price')

  # Sort DataFrame based on detected keywords
  sorted_df = df.sort_values(by=sort_criteria, ascending=[False if col == 'efficiency' else True for col in sort_criteria])

  # Return the top 3 products based on the sorting criteria
  return sorted_df[['title', 'discount_price', 'efficiency']].head(3)

sorted_products = filter_and_sort_data(df, detected_keywords)
print(sorted_products)


                                                  title  discount_price  \
1381  【SANLUX 台灣三洋】福利品2-3坪5級定頻分離式冷專冷氣(SAE-22S1/SAC-2...           12590   
310   【MIDEA 美的】3-5坪 L系列超值 變頻冷專 2.9kW分離式空調(MVC-L28CA...           12990   
1266    【MIDEA 美的】L系列4-5坪冷專變頻分離式冷氣(MVC-L28CA/MVS-L28CA)           12990   

     efficiency  
1381    Level 5  
310     Level 5  
1266    Level 5  


In [10]:
def get_ask_brand(reviews_dict, brand_name, prompt):
  ask_brand = ''
  for key in reviews_dict:
    if key in prompt.upper():
      ask_brand = key
      break
  if not ask_brand:
    for i in range(len(brand_name)):
      if brand_name.CH[i] in prompt:
        ask_brand = brand_name.ENG[i]
  return ask_brand

In [11]:
def get_abstract(reviews_dict, ask_brand):
  client2 = openai.Client(api_key=APIKEY)
  model = 'gpt-4o-mini'
  messages_simple = []
  messages_simple.append({"role": "system", "content": f"你負責將使用者主觀的評價整理成客觀的摘要。"})
  messages_simple.append({"role": "assistant", "content": f"請將{reviews_dict[ask_brand]}的value整理成150字以內的摘要。"})

  result = client2.chat.completions.create(
      model=model,
      messages = messages_simple
      )

  return result.choices[0].message.content

In [12]:
reviews_dict = {}

for i in range(len(reviews_momo.brand)):
  if reviews_momo.brand[i] not in reviews_dict:
    reviews_dict[reviews_momo.brand[i]] = reviews_momo.comment[i]
  else:
    reviews_dict[reviews_momo.brand[i]] += reviews_momo.comment[i]
# print(reviews_dict)

In [13]:
test_brand = '東元'
summary = get_abstract(reviews_dict, get_ask_brand(reviews_dict, brand_name, test_brand))
print(summary)

使用者對東元冷氣的評價呈現兩極化趨勢。他們讚揚東元的高性價比、省電和安靜運行，尤其是HS系列的舒適度以及售後服務的改善。然而，部分使用者則對冷氣的耐用性表示擔憂，指出主機板故障及維修困難，並對售後服務體驗表示不滿。總體而言，東元在性價比與技術進步上受到認可，但仍需加強產品的可靠性和客戶服務。


In [14]:
import openai

# Example usage
api_key = APIKEY
prompt = "User query: 我想要知道能效高和價格便宜的空調"

assistant_message, messageList = chat_with_openai(prompt, [])

# Function to call OpenAI API
def get_user_query(api_key, prompt):
    openai.api_key = api_key
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messageList,
        max_tokens=150
    )
    return response.choices[0].message.content

# # Example usage
# api_key = APIKEY
# prompt = "User query: 我想要知道能效高和價格便宜的空調"
user_query = get_user_query(api_key, prompt)

# Parse the user's query
detected_keywords = parse_user_query(user_query)

# Filter and sort the data
sorted_products = filter_and_sort_data(df, detected_keywords)

# Display the result
print(sorted_products)


                                                  title  discount_price  \
1381  【SANLUX 台灣三洋】福利品2-3坪5級定頻分離式冷專冷氣(SAE-22S1/SAC-2...           12590   
310   【MIDEA 美的】3-5坪 L系列超值 變頻冷專 2.9kW分離式空調(MVC-L28CA...           12990   
1266    【MIDEA 美的】L系列4-5坪冷專變頻分離式冷氣(MVC-L28CA/MVS-L28CA)           12990   

     efficiency  
1381    Level 5  
310     Level 5  
1266    Level 5  


In [15]:
def main():
  messageList = [
    {"role": "system", "content": f"你是根據使用者需求推薦適合冷氣的業務。"}
  ]

  with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    state = gr.State(messageList)

    with gr.Row():
      text = gr.Textbox(
        show_label=False,
        placeholder="請輸入對於冷氣所期望的條件，像是房間坪數、省電能效、預算限制、保固期間等。如想了解之品牌評價，請輸入品牌名稱以及「評價」二字。"
      )

    def respond(user_input, history, state):
      response, new_state = handle_chat(user_input, state, df)
      history.append((user_input, response))
      return history, new_state

    text.submit(respond, [text, chatbot, state], [chatbot, state])

  demo.launch(share=True)

if __name__ == '__main__':
    main()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://643feef2417039b06f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# def greet(name):
#     return "Hello " + name


# demo = gr.Interface(fn=greet, inputs="text", outputs="text")

# demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2bed5370ac164f2605.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
